This program give information on how much one will need to retire.

In [4]:
import pandas as pd
import numpy as np
import time

In [29]:
###################################################
# METHODS
###################################################

def init_df():
    df = pd.DataFrame(columns= ['APR', 'Years', 'Starting Balance', 'Ending Balance'])
    return df

def impact_years_compounding(df, how_much_saved, yrs):
    
    yr = 1
    while yr <= yrs:
                  
    
        df = df.append({'APR': APR, 'Years': Years, 'Starting Balance': S, 'Ending Balance': B} , ignore_index = True)
    
        return df
    

def impact_range_of_annual_percentage_rates(how_much_saved):
    apr = 1 # ir = annual interest rate (APR)
    while apr in range(50): #cycles through different interest rates from 1 to 30.
        
        final_principle = annual_by_month_compound(how_much_saved,apr) #ending principle after 1 year
        print('Your starting principle was: ', how_much_saved)
        print('The annual interest rate is: ', apr, '%')
        print('The annualized final principle amortized monthly is: ', final_principle, '\n')
        apr += 1
        
def annual_by_month_compound(how_much_saved, ir):
    irm = ir/1200 # APR divided by 12 to get monthly interest rate
    print('irm = ', irm)
    month = 1    
    new_principle = float(how_much_saved)
    print('new_principle = ', new_principle, '\n')
    while month < 13:
        gain_from_interest_on_principle = new_principle * irm
        #print('gain_from_interest_on_principle =', gain_from_interest_on_principle)
        new_principle = new_principle + gain_from_interest_on_principle
        month += 1
    return new_principle



In [30]:
#########################################
# MAIN #
#########################################

# Variable definition

'''years_to_retire = input('How many years to retirement? ')
how_much_needed = input('How much annual income will you need in retirement? ')
how_much_saved = input('How much have you saved for retirement? ')
how_much_ss = input('How much montly social security will you be getting? ')
how_much_other_passive_income = input('How much other monthly passive income will you be getting? \n')
'''

years_to_retire = 5
how_much_needed = 50
how_much_saved = 100
how_much_ss = 2
how_much_other_passive_income = 0


impact_range_of_annual_percentage_rates(how_much_saved)



# Results

irm =  0.0008333333333333334
new_principle =  100.0 

Your starting principle was:  100
The annual interest rate is:  1 %
The annualized final principle amortized monthly is:  101.0045960887182 

irm =  0.0016666666666666668
new_principle =  100.0 

Your starting principle was:  100
The annual interest rate is:  2 %
The annualized final principle amortized monthly is:  102.01843556815014 

irm =  0.0025
new_principle =  100.0 

Your starting principle was:  100
The annual interest rate is:  3 %
The annualized final principle amortized monthly is:  103.04159569135074 

irm =  0.0033333333333333335
new_principle =  100.0 

Your starting principle was:  100
The annual interest rate is:  4 %
The annualized final principle amortized monthly is:  104.07415429197894 

irm =  0.004166666666666667
new_principle =  100.0 

Your starting principle was:  100
The annual interest rate is:  5 %
The annualized final principle amortized monthly is:  105.11618978817332 

irm =  0.005
new_principle =  10

In [53]:
df = pd.DataFrame(columns= ['APR', 'Years', 'Starting Balance', 'Ending Balance'])
list(df.columns)

['APR', 'Years', 'Starting Balance', 'Ending Balance']

In [54]:
i = 0

while i < 5:
        
    APR = i
    Years = i
    S = i
    B = i 
    
    df = df.append({'APR': APR, 'Years': Years, 'Starting Balance': S, 'Ending Balance': B} , ignore_index = True)

    i += 1
        
print(df)

  APR Years Starting Balance Ending Balance
0   0     0                0              0
1   1     1                1              1
2   2     2                2              2
3   3     3                3              3
4   4     4                4              4


In [46]:
print(df)

Empty DataFrame
Columns: [APR, Years, Starting Balance, Ending Balance]
Index: []


In [49]:
df = pd.DataFrame(columns=['A'])
for i in range(5):
    df = df.append({'A': i}, ignore_index=True)
df

,A
0,0
1,1
2,2
3,3
4,4
